# Code

## Установка библиотек и функций

In [ ]:
#@title Установка и импорт библиотек
!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 faiss-cpu==1.7.4 gspread oauth2client nltk pydantic==1.10.8 openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uninstalling pydantic-1.10.13:
      Successfully uninstalled pydantic-1.10.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not inst

In [ ]:
import gdown
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
#database
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter
import re
#from langchain.document_loaders import TextLoader

from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import getpass
import os
import openai
import tiktoken
from io import BytesIO
import openpyxl
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
import os
import platform

def clear_screen():
    system_platform = platform.system()  # определить операционную систему
    if system_platform == "Windows":
        os.system('cls')
    else:
        os.system('clear')

In [ ]:
MODEL_GPT_3_5_TURBO_16K = ['gpt-3.5-turbo-1106', 0.001, 0.002]  # 16K context window, [1], [2] - price per 1k tokens
MODEL_GPT_3_5_TURBO = ['gpt-3.5-turbo-0613', 0.0015, 0.002]  # 4,096 tokens, Shutdown date 2024-06-13
MODEL_GPT_3_5_TURBO_INSTRUCT = ['gpt-3.5-turbo-instruct', 0.0015, 0.002]  # 4,096 tokens
MODEL_GPT_4 = ['gpt-4', 0.03, 0.06]  # 8,192 tokens
MODEL_GPT_4_TURBO = ['gpt-4-1106-preview', 0.01, 0.03]  # 128k tokens	content
SELECT_MODEL_GPT = MODEL_GPT_3_5_TURBO_16K # выбери нужную модель

In [ ]:
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Returns the number of tokens in a text string."""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

In [ ]:
def insert_newlines(textstr: str, max_len: int = 150) -> str:
    words = textstr.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

In [ ]:
#@title функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
#@title  функция для загрузки txt файла по ссылке
def load_text(url: str) -> str:
    # Download the document as plain text
    response = requests.get(url)
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
#@title  Функция загрузки базы знаний из файла и создания векторной базы
def load_file_knowledge(file_path: str) -> str:
    # Чтение текстового файла
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    headers_to_split_on = [
        ("#", "Header1"),
        ("##", "Header2"),
        ("###", "Header3"),
        ("####", "Header4"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    md_header_splits = markdown_splitter.split_text(text)

    # Предполагается, что FAISS и OpenAIEmbeddings были импортированы или определены где-то выше
    vectordateBase = FAISS.from_documents(md_header_splits, OpenAIEmbeddings())   # ada v2	$0.0001 / 1K tokens
    vectordateBase.save_local("/content/faiss")   # сохраняем векторную базу локально

    count_token = num_tokens_from_string(' '.join([x.page_content for x in md_header_splits]), "cl100k_base")
    print('Количество токенов в документе :', count_token)
    print('ЦЕНА запроса:', 0.0001 * (count_token / 1000), ' $')

    return vectordateBase



In [ ]:
#@title Функция загрузки векторной базы из папки
def load_vectordate_base(path_to_vectordate_base: str) -> str:

    vectordate_base = FAISS.load_local(path_to_vectordate_base, OpenAIEmbeddings())
    return vectordate_base

In [ ]:
#@title функция суммаризации диалогов
def _summarize_topic(dialog):
        messages = [
            {"role": "system",
             "content": "Ты - ассистент консультанта, основанный на AI. Ты умеешь профессионально суммаризировать присланные тебе диалоги консультанта и клиента. Твоя задача - суммаризировать диалог, который тебе пришел."},
            {"role": "user",
             "content": "Суммаризируй следующий диалог консультанта и клиента: " + " ".join(dialog)}
        ]

        completion = openai.ChatCompletion.create(
            model=SELECT_MODEL_GPT[0],
            messages=messages,
            temperature=0.1,  # Используем более низкую температуру для более определенной суммаризации
            max_tokens=1000  # Ограничиваем количество токенов для суммаризации
        )

        return completion.choices[0].message.content

### Загрузка таблицы для тестирования

In [ ]:
# функция для загрузки таблицы по ссылке из гугл драйв
#def load_xls_pd(url: str) -> str:
def load_xls_pd(url: str, sheet_name: str = 0) -> str:

    # Extract the document ID from the URL
    match_ = re.search('/spreadsheets/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Sheets URL')
    doc_id = match_.group(1)

    if sheet_name is None:
        raise ValueError('Invalid sheet_name')

    # Download the table as pandas
    response = requests.get(f'https://docs.google.com/spreadsheets/d/{doc_id}/export?format=xlsx')
    response.raise_for_status()     #проверяет статус код ответа. Если получен ответ с кодом ошибки (4xx или 5xx), вызывается исключение HTTPError.
    #data = pd.read_excel(BytesIO(response.content), sheet_name=0)  #чтение первого листа из книги
    data = pd.read_excel(BytesIO(response.content),sheet_name = sheet_name, header=None) #чтение конкретного листа из Книги Excel

    ''' Разъяснение:
        Когда мы хотим прочитать данные Excel с помощью функции pd.read_excel(), она требует передачи ей пути к файлу или объекта, представляющего файл.
        В параметре io функции pd.read_excel() необходимо указать путь к файлу (в виде строки, содержащей путь к файлу) или объект файлового типа (такой как BufferedWriter, BufferedReader и другие).
        response.content возвращает содержимое ответа на запрос HTTP в виде байтового массива (bytes array).
        Чтобы передать эти данные в функцию pd.read_excel(), нужно создать объект файла из байтового массива.
        Для этой цели используется объект BytesIO из модуля io, который предоставляет интерфейс для работы с данными в памяти, как если бы они находились в файле.
    '''
    return data

In [ ]:
# Шаг 1: Загрузить xlsx файл из Google Drive
google_sheet_url = "https://docs.google.com/spreadsheets/d/1ewjENXCLwfmzGA9FFTqb6lcg4Ob9N505Lg-4H8oQvoI"
sheet_name='Вопросы с 10.11'

#data = load_xls_pd(google_sheet_url)   #если в таблице только один лист
df = load_xls_pd(google_sheet_url, sheet_name)

In [ ]:
# проверка
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,Тестировщик,Раздел,Тема,Вопрос,Версия GPT,Ответ GPT,Ожидаемый ответ (не обяз. нужен для дальнейшег...,Категория неудачных ответов,Комментарий к оценке,Оценка -2...+2,Чанки,NaN
1,Руслан Г.,models,Kia Sportage,NaN,GPT 3.5,NaN,NaN,NaN,NaN,NaN,NaN,1.409574
2,Руслан Г.,models,Kia Sportage,Какие фишки у Sportage,GPT 3.5,NaN,NaN,NaN,Списки оформлены не под телеграм,NaN,NaN,NaN
3,Руслан Г.,models,Kia Sportage,"Какие технологии используются в Sportage, чтоб...",GPT 3.5,NaN,NaN,NaN,NaN,2,NaN,NaN
4,Руслан Г.,models,Kia Sportage,Какие преимущества предлагает Kia Sportage в с...,GPT 3.5,NaN,NaN,NaN,NaN,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
606,Татьяна,NaN,Service Maintenance,"Если ТО делают при пробеге 30000 км?2. ""Зачем ...",GPT 3.5,Понял вас. При пробеге 30000 км рекомендуется ...,NaN,NaN,NaN,2,NaN,NaN
607,Татьяна,NaN,Service Maintenance,Просьба уточнить гарантия на подвеску распрост...,GPT 3.5,Подвеска автомобиля входит в категорию узлов и...,NaN,NaN,"Вопрос скорее относится к гарантии, а не к сер...",1,NaN,NaN
608,Галина,NaN,ПНД,"Как оформить услугу ""Помощь на дорогах"".",GPT 3.5,NaN,NaN,NaN,а здесь корректно определи роутер (так как был...,2,NaN,NaN
609,Галина,NaN,ПНД,Как подключить Помощь на дорогах.,GPT 3.5,NaN,NaN,NaN,"роутер определил некорректно (apps), но там на...",2,NaN,NaN


In [ ]:
# установить первую строку в виде строки заголовков столбцов
df.set_axis(df.iloc[0], axis='columns', inplace=True)
# проверка
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 12 columns):
 #   Column                                                             Non-Null Count  Dtype  
---  ------                                                             --------------  -----  
 0   Тестировщик                                                        411 non-null    object 
 1   Раздел                                                             150 non-null    object 
 2   Тема                                                               611 non-null    object 
 3   Вопрос                                                             609 non-null    object 
 4   Версия GPT                                                         387 non-null    object 
 5   Ответ GPT                                                          183 non-null    object 
 6   Ожидаемый ответ (не обяз. нужен для дальнейшего автотестирования)  27 non-null     object 
 7   Категория неудачных ответо

<ipython-input-17-1525854632f0>:2: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(df.iloc[0], axis='columns', inplace=True)


,Тестировщик,Раздел,Тема,Вопрос,Версия GPT,Ответ GPT,Ожидаемый ответ (не обяз. нужен для дальнейшего автотестирования),Категория неудачных ответов,Комментарий к оценке,Оценка -2...+2,Чанки,NaN
0,Тестировщик,Раздел,Тема,Вопрос,Версия GPT,Ответ GPT,Ожидаемый ответ (не обяз. нужен для дальнейшег...,Категория неудачных ответов,Комментарий к оценке,Оценка -2...+2,Чанки,NaN
1,Руслан Г.,models,Kia Sportage,NaN,GPT 3.5,NaN,NaN,NaN,NaN,NaN,NaN,1.409574
2,Руслан Г.,models,Kia Sportage,Какие фишки у Sportage,GPT 3.5,NaN,NaN,NaN,Списки оформлены не под телеграм,NaN,NaN,NaN
3,Руслан Г.,models,Kia Sportage,"Какие технологии используются в Sportage, чтоб...",GPT 3.5,NaN,NaN,NaN,NaN,2,NaN,NaN
4,Руслан Г.,models,Kia Sportage,Какие преимущества предлагает Kia Sportage в с...,GPT 3.5,NaN,NaN,NaN,NaN,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
606,Татьяна,NaN,Service Maintenance,"Если ТО делают при пробеге 30000 км?2. ""Зачем ...",GPT 3.5,Понял вас. При пробеге 30000 км рекомендуется ...,NaN,NaN,NaN,2,NaN,NaN
607,Татьяна,NaN,Service Maintenance,Просьба уточнить гарантия на подвеску распрост...,GPT 3.5,Подвеска автомобиля входит в категорию узлов и...,NaN,NaN,"Вопрос скорее относится к гарантии, а не к сер...",1,NaN,NaN
608,Галина,NaN,ПНД,"Как оформить услугу ""Помощь на дорогах"".",GPT 3.5,NaN,NaN,NaN,а здесь корректно определи роутер (так как был...,2,NaN,NaN
609,Галина,NaN,ПНД,Как подключить Помощь на дорогах.,GPT 3.5,NaN,NaN,NaN,"роутер определил некорректно (apps), но там на...",2,NaN,NaN


In [ ]:
#df['Чанки'] = pd.Series(dtype=str)   # так можно добавить столбец "Чанки", если его нет

In [ ]:
#df = df.drop('Чанки', axis=1)         # а так можно удалить столбец

In [ ]:
#df.insert(loc=10, column='Чанки', value='') #2ой способ добавить пустой столбец "Чанки", если он отсутствует 11 столбцом

## Загружаем векторное хранилище либо из базы знаний:
Предварительно нужно загрузить файл базы знаний 'database.md' в колаб

In [ ]:
#Если заново собирать векторную базу из базы знаний, то нужно раскоментировать этот код
'''
!git clone -b develop https://github.com/TasksAndreySorokin/KIA-GPT1.git
!cp /content/KIA-GPT1/knowledge/database.md /content/database.md
!rm -r /content/KIA-GPT1
knowledge_base = load_file_knowledge('database.md')
'''

Cloning into 'KIA-GPT1'...
remote: Enumerating objects: 1884, done.
remote: Counting objects: 100% (902/902), done.
remote: Compressing objects: 100% (321/321), done.
remote: Total 1884 (delta 676), reused 724 (delta 576), pack-reused 982
Receiving objects: 100% (1884/1884), 58.07 MiB | 26.01 MiB/s, done.
Resolving deltas: 100% (1022/1022), done.
Количество токенов в документе : 656705
ЦЕНА запроса: 0.0656705  $


In [ ]:
#!zip -r faiss.zip faiss     #для архивирования папки, чтобы ее можно было скачать себе

## Либо используем уже готовую векторную базу (два раза НЕ НУЖНО):
Предварительно загрузим папку FAISS_routing с векторными базами в колаб и скопируем ее в нужное место:

In [ ]:
!git clone -b develop https://github.com/TasksAndreySorokin/KIA-GPT1.git
!cp -r /content/KIA-GPT1/knowledge/faiss /content/faiss
!rm -r /content/KIA-GPT1

Cloning into 'KIA-GPT1'...
remote: Enumerating objects: 2019, done.
remote: Counting objects: 100% (1034/1034), done.
remote: Compressing objects: 100% (372/372), done.
remote: Total 2019 (delta 760), reused 854 (delta 656), pack-reused 985
Receiving objects: 100% (2019/2019), 70.76 MiB | 21.49 MiB/s, done.
Resolving deltas: 100% (1109/1109), done.


In [ ]:
#!unzip /content/faiss.zip   #можно загрузить в колаб из локальной  папки с векторным хранилищем и разархиваровать

## Функция запроса к ChatGPT

In [ ]:
# Подгружаем векторное хранилище
knowledge_base = load_vectordate_base('faiss')

In [ ]:
def answer_index(topic, temp=0.1, top_similar_documents=10):
        summarize_text = ''
        HISTORY = [] # ОТКЛЮЧАЕТ САММАРИЗАЦИЮ! ЕСТЬ НУЖНО ЕЕ ВКЛЮЧИТЬ ТОГДА ЗАКОММЕНТИРУЙ
        if len(HISTORY) > 0:
            summarize_text = "Вот краткий обзор предыдущего диалога: " + _summarize_topic(
                [q + ' ' + (a if a is not None else '') for q, a in HISTORY])
        #print(f'САММАРИ \n=== {insert_newlines(summarize_text)} \n')

        # Добавляем явное разделение между историей диалога и текущим вопросом
        input_text = summarize_text + "\n\nТекущий вопрос: " + topic

        docs = knowledge_base.similarity_search_with_score(topic, k=top_similar_documents)

        responses = []
        for i, (doc, score) in enumerate(docs):
            if score < 2: # ТУТ ТЫ МОЖЕШЬ УПРАВЛЯТЬ праметром Л2 для чанков. 0..1
                content = doc.page_content
                response = f' ___ score = {score} ___ Metadata документа = {doc.metadata}\n___Отрывок документа №{i + 1}___\n{content}\n'

                # раскомментируй строки ниже для вывода на экран найденных чанков
                # print(f'\n=====================Отрывок документа №{i + 1}=====================\n')
                # print(f'=== score = {score}  Metadata документа ------------ {doc.metadata}')
                # print(f'\n{content}\n')

                responses.append(response)

        messages = [
            {"role": "system", "content": prompt},
            {"role": "user",
             "content": f"Документ с информацией для ответа клиенту: {responses}\n\nВопрос клиента: \n{input_text}"}
        ]

        completion = openai.ChatCompletion.create(
            model=SELECT_MODEL_GPT[0],
            messages=messages,
            temperature=temp
        )

        print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
        print('ЦЕНА запроса с ответом :', (SELECT_MODEL_GPT[1])*(completion["usage"]["prompt_tokens"]/1000) + (SELECT_MODEL_GPT[2])*(completion["usage"]["completion_tokens"]/1000), ' $')
        print('===========================================: \n')

        answer = completion.choices[0].message.content

        # Добавляем вопрос пользователя и ответ системы в историю
        HISTORY.append((topic, answer if answer is not None else ''))

        return  insert_newlines(answer), responses

In [ ]:
prompt = load_text('https://raw.githubusercontent.com/TasksAndreySorokin/KIA-GPT1/develop/Prompts/kia_main_prompt.txt')

## Для обработки диалогов вручную

In [ ]:
def run_dialog():

    while True:
        user_question = input('\nКлиент: ')
        if ((user_question.lower() == 'stop') or (user_question.lower() == 'стоп')):
            break
        answer, chanks = answer_index(user_question)
        print('\nМенеджер: ', answer + '\n\n')

    return


In [ ]:
# закоментируй для сохранения ответов и чанков в таблицу

verbose=1
HISTORY = [] # Убери решетку если нужно обновлять историю диалогов после запуска, если не нужно обновлять то оставь так

run_dialog()


4218 токенов использовано всего (вопрос-ответ).
ЦЕНА запроса с ответом : 0.004261  $
===========================================: 


Менеджер:   Извините, я могу уточнить, для какой модели автомобиля вам нужны диски 16 дюймов 215?


4336 токенов использовано всего (вопрос-ответ).
ЦЕНА запроса с ответом : 0.00439  $
===========================================: 


Менеджер:   Извините, я могу уточнить, для какой модели и конфигурации автомобиля вам нужны диски 16 дюймов 215/60R16?


4305 токенов использовано всего (вопрос-ответ).
ЦЕНА запроса с ответом : 0.004324  $
===========================================: 


Менеджер:   Извините, я не могу ответить на ваш вопрос.


4343 токенов использовано всего (вопрос-ответ).
ЦЕНА запроса с ответом : 0.004403000000000001  $
===========================================: 


Менеджер:   Извините, я могу уточнить, для какой модели и конфигурации автомобиля вам нужна информация о цене дисков 16 дюймов 215/60R16?


4329 токенов использовано всего (вопро

KeyboardInterrupt: ignored

In [ ]:
# Проверка истории
#HISTORY

## Сохранение ответов ChatGPT и чанков в таблицу

In [ ]:
# Переменная для подсчета количества строк, обработанных по условию
count = 0
print('Количество обработанных запросов: ')
# Цикл для обхода строк DataFrame
for index, row in tqdm(df.iterrows()):
    if (row['Тема'] == 'Аксесуары') & (pd.isna(row['Ответ GPT'])):       # Выбираем строки со своей Темой для тестирования, и если не пустые
        # Ваш код для обработки данных текущей строки
        user_question = row['Вопрос']
        answer, chanks = answer_index(user_question)

        # Записываем результат в столбец "Ответ"
        df.at[index, 'Ответ GPT'] = answer
        # Записываем чанки
        df.at[index, 'Чанки'] = chanks
        # Увеличиваем значение счетчика и проверяем, достигли ли мы нужного количества запросов
        count += 1
        print(count, end=' ')
        if count >= 6:          # количество задаваемых вопросов к ChatGPT
            break


Количество обработанных запросов: 


0it [00:00, ?it/s]

4300 токенов использовано всего (вопрос-ответ).
ЦЕНА запроса с ответом : 0.0043230000000000005  $
===========================================: 

1 4077 токенов использовано всего (вопрос-ответ).
ЦЕНА запроса с ответом : 0.004176000000000001  $
===========================================: 

2 4146 токенов использовано всего (вопрос-ответ).
ЦЕНА запроса с ответом : 0.004231  $
===========================================: 

3 3847 токенов использовано всего (вопрос-ответ).
ЦЕНА запроса с ответом : 0.0039030000000000002  $
===========================================: 

4 5634 токенов использовано всего (вопрос-ответ).
ЦЕНА запроса с ответом : 0.0057090000000000005  $
===========================================: 

5 6600 токенов использовано всего (вопрос-ответ).
ЦЕНА запроса с ответом : 0.006706  $
===========================================: 

6 

In [ ]:
df[df['Тема'] == 'Аксесуары'][['Вопрос', 'Ответ GPT', 'Чанки']].head(15)   #проверка записей в таблице

,Вопрос,Ответ GPT,Чанки
301,Есть ли диски 16 дюймов,Какая модель и конфигурация автомобиля вас ин...,[ ___ score = 0.2914389371871948 ___ Metadata ...
302,Есть ли диски 14,"К сожалению, у меня нет информации о наличии ...",[ ___ score = 0.32527098059654236 ___ Metadata...
303,Есть ли коврики для GT Linе,"Да, для комплектации GT Line доступны коврики...",[ ___ score = 0.27616971731185913 ___ Metadata...
304,Есть ли колпаки для диска для комплектации Com...,"Извините, я могу узнать модель и конфигурацию...",[ ___ score = 0.26951855421066284 ___ Metadata...
305,Есть ли защита картера,"Я вижу, что вы интересуетесь защитой картера....",[ ___ score = 0.26443806290626526 ___ Metadata...
306,Можно ли отдельно купить систему помощи при па...,"Да, вы можете отдельно купить систему помощи ...",[ ___ score = 0.2579635977745056 ___ Metadata ...


In [ ]:
df[(df['Тема'] == 'Аксесуары') & (df['Ответ GPT'].isna())].head(10) #проверка записей в таблице

,Тестировщик,Раздел,Тема,Вопрос,Версия GPT,Ответ GPT,Ожидаемый ответ (не обяз. нужен для дальнейшего автотестирования),Категория неудачных ответов,Комментарий к оценке,Оценка -2...+2,Чанки,NaN


In [ ]:
df.to_excel('test.xlsx')      # загрузка df в таблицу 'test.xlsx'

Далее:
- в исходной таблице установить фильтр для отображения только своего раздела, чтобы не перезаписать чужие разделы.
- открыть созданную таблицу в Гугл Таблицах, установить фильтр для отображения только своего раздела, чтобы не перезаписать чужие разделы.
- скопировать по очереди целиком столбцы 'Ответ GPT' и 'Чанки' в исходную таблицу для тестирования.